In [7]:
import numpy as np

## Stage 1: Bomber Builder

Goals:
Practice Python
    Specifically, arrays, numpy, and  objects/classes
Build a bomber object that can be shot at, and then have armor added to it.

Okay, in order to armor a bomber I need to shoot a bomber, in order to shoot a bomber I need to build it. What I'm imagining right now is a three dimensional array representing the three dimensional 'space' of the bomber. Each cell would represent either a part of the bomber, or the air around the bomber. Each part of the 'bomber' would have a value representing how well that part of the bomber responds to having holes put into it. 

Right now I can see two ways of modeling that. 

I can put an integer value into each cell, with higher values representing more ability to take a hit. Then a hit in this spot would iterate that number down. Once the value hits zero, the plane is down. Armor would be modeled as a layer of numbers put on top of the base array. Call this the Ablative Model.

Another way to do this would be to assign each cell a value between 0 and 1, representing the probability that, given a cell is struck, will it cripple the plane. Armor in this model would be represented by a layer that increases the odds of survival for a cell, with as a flat modifier or some sort of multiplicative effect. Call this the Probabalistic Model. 

I want to build something that's flexible enough to take both options. Or even, could 

Right now I'm picturing a 3x25x25 (or 25x25x3) array. That's big enough to make a plane, small enough that I can picture it. Now, the math doesn't particularly care, but I'd find it easy to, at least at first, make a vaugely plane shaped plane. So wings, a cockpit, and a tail.

In [10]:

blankarray = np.empty((25,25,3)) 

In [11]:
blankarray

array([[[ 6.88809390e-312,  6.88810770e-312,  0.00000000e+000],
        [-3.72477755e+065,  6.88996237e-312,  6.88996238e-312],
        [ 5.95203902e-226,  0.00000000e+000,  0.00000000e+000],
        ...,
        [-1.00757526e+248,  6.88996435e-312,  6.88996435e-312],
        [ 2.91451745e+131,  0.00000000e+000,  0.00000000e+000],
        [ 2.91451745e+131,  6.88996432e-312,  6.88996449e-312]],

       [[-1.08234902e-172,  6.88996429e-312,  6.88996429e-312],
        [-3.93666710e+266,  0.00000000e+000,  0.00000000e+000],
        [-3.93666710e+266,  6.88996436e-312,  6.88996450e-312],
        ...,
        [ 1.48375364e+031,  6.88996487e-312,  6.88996498e-312],
        [ 1.10338708e+044,  6.88996438e-312,  6.88996477e-312],
        [ 1.27972843e+297,  6.88796892e-312,  6.88812554e-312]],

       [[ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
        [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
        [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
        ..

In [ ]:




# I 'pseudocoded' this on a piece of paper. It's a good thing I didn't go into visual arts.
